In [ ]:
!pip install --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara
!pip install implicit

  Cloning https://github.com/evfro/polara.git (to revision develop) to /tmp/pip-install-yobrcil3/polara_c224795b242c4938bffd152c36b1de8d
  Running command git clone --filter=blob:none --quiet https://github.com/evfro/polara.git /tmp/pip-install-yobrcil3/polara_c224795b242c4938bffd152c36b1de8d
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Resolved https://github.com/evfro/polara.git to commit 8fdb520722f4c80c20709dbbda5b0896c4ca6f7a
  Preparing metadata (setup.py) ... done
  Created wheel for polara: filename=polara-0.7.2.dev0-py3-none-any.whl size=90560 sha256=cf6b13a0a5d2de25cef067c984a985c6ccc63107931b5f425769a5c9726f53fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-9pn3h0lh/wheels/a3/85/5c/a60c513136efd6e59772f161f65588683e0ef52481187ebb5a
Successfully built polara


In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=938107 sha256=c2b7c83110d9b0ec5def199ca6f5f95a2f53b86c7a7ecdcc71e66defc14eb998
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


In [ ]:
import numpy as np
from polara.datasets.movielens import get_movielens_data

In [ ]:
seed = 0
def random_state(seed=seed): # to fix random state in experiments
    return np.random.RandomState(seed=seed)

In [ ]:
random_state()

RandomState(MT19937) at 0x7E0BFB4CD040

In [ ]:
data = get_movielens_data(include_time=True)

In [ ]:
data = data.sort_values("timestamp").reset_index(drop=True)

In [ ]:
split_date = data["timestamp"].quantile(0.95)

data_split = dict()

#train data
data_split["train"] = data[data["timestamp"] <= split_date]

#test
data_split["test"] = (
    data[data["timestamp"] > split_date]
    .groupby("userid")
    .apply(lambda x: x.iloc[:-1])
    .reset_index(drop=True)
)

#holdout
data_split["holdout"] = (
    data[data["timestamp"] > split_date]
    .groupby("userid")
    .apply(lambda x: x.iloc[-1])
    .reset_index(drop=True)
)
# This data split, combined with `warm_start=False` in `data_model.set_test_data`,
# configures the evaluation for cold-start scenarios, specifically for items.
# All users in the `holdout` set have prior interactions in the `train` set (warm users).

/tmp/ipython-input-1432808225.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.iloc[:-1])
/tmp/ipython-input-1432808225.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.iloc[-1])


In [ ]:
from polara.recommender.data import RecommenderData
from polara.recommender.models import SVDModel, PopularityModel, RandomModel, ProbabilisticMF
from polara.recommender.external.implicit.ialswrapper import ImplicitALS

In [ ]:
data_model = RecommenderData(data_split["train"], 'userid', 'movieid', 'rating', seed=seed)
data_model.prepare_training_only()
data_model.set_test_data(testset=data_split["test"], holdout=data_split["holdout"], warm_start=True)

Preparing data...
Done.
There are 950198 events in the training and 0 events in the holdout.
17 unique movieid entities within 24 testset interactions were filtered. Reason: not in the training data.
1 unique movieid entities within 1 holdout interactions were filtered. Reason: not in the training data.
34 userid entities were filtered out from holdout. Reason: inconsistent with testset.
1 userid entities were filtered out from testset. Reason: inconsistent with holdout.
Done. There are 778 events in the holdout.


/usr/local/lib/python3.12/dist-packages/polara/recommender/data.py:704: FutureWarning: DataFrameGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  grouper = data.groupby(col, sort=sort).grouper
/usr/local/lib/python3.12/dist-packages/polara/recommender/data.py:704: FutureWarning: DataFrameGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  grouper = data.groupby(col, sort=sort).grouper


In [ ]:
svd = SVDModel(data_model)
popular = PopularityModel(data_model)
random = RandomModel(data_model, seed=0)
als = ImplicitALS(data_model)
models = [svd, popular, random, als]

In [ ]:
models[0].get_base_configuration()

{'topk': 10,
 'filter_seen': True,
 'switch_positive': None,
 'feedback_threshold': None,
 'verify_integrity': True}

In [ ]:
for i in models:
  i.build()

PureSVD training time: 0.241s


/usr/local/lib/python3.12/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.014693975448608398 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

iALS training time: 4.920s


In [ ]:
"""
исправить + перезапустить ноут
/usr/local/lib/python3.12/dist-packages/polara/recommender/evaluation.py
Добавь параметр topn в сигнатуру get_ndcr_discounts.

Раскомментируй 2 строки (ideal_indices и idx) и закомментируй текущие версии
/usr/local/lib/python3.12/dist-packages/polara/recommender/external/impilicit/ialswrapper.py
иcправить строчки 78,79 на
recs = self._model.recommend(user_row, matrix[user_row], N=self.topk, recalculate_user=recalculate)
top_recs[i, :] = recs[0]
"""

In [ ]:
for m in models:
    res = m.evaluate()          # [Relevance(...), Ranking(...), Experience(...), Hits(...)]
    ranking = res[1]
    experience = res[2]
    hits = res[3]

    hitrate = hits.true_positive / (hits.true_positive + hits.false_negative)

    print(f"{m.method:8s} ndcg={ranking.ndcg:.6g}  coverage={experience.coverage:.6g}  hitrate={hitrate:.6g}")


PureSVD  ndcg=0.0130714  coverage=0.0889371  hitrate=0.0257069
MP       ndcg=0.00493089  coverage=0.0103037  hitrate=0.00899743
RND      ndcg=0.00153559  coverage=0.873915  hitrate=0.00385604
iALS     ndcg=0.000810964  coverage=0.257863  hitrate=0.00128535


{'topk': 10,
 'filter_seen': True,
 'switch_positive': None,
 'feedback_threshold': None,
 'verify_integrity': True}